# Evaluating non-coherent averaging detection performance
This notebook does the following:
1) Generates spectrogram samples (noise-only or tone+noise)
2) Performs non-coherent averaging ranging from 2 to 512 line averages
3) Evaluates effect of DC offsets and non-flat PFB frequency response on mean, std and threshold setting, and ultimate non-coherent averaging SNR gain
4) Applies split-window-two-pass-mean (sw2pm) algorithm as a means of removing the effect of PFB frequency response.

Note: does not evaluate simple multi-band normalizer "simple_norm.py" alternative to sw2pm

In [ ]:
import sys
import os

#matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import scipy as sp
from scipy.fft import fft, fftshift

import time

from pathlib import Path
from numpy.random import default_rng

from sw2pm import sw2pm
from gen_simple_coarse_sg import gen_simple_coarse_sg

from numpy.random import randn, rand, randint, seed
seed(22)

output_dir = './noncoh_plots/'
if not os.path.isdir(output_dir[0:-1]):
    os.system('mkdir '+output_dir[0:-1])

def db(x):
    """ Convert linear value to dB value, excludes x as list """
    return 10.*np.log10(abs(x)+1e-20)

def rms(x,axis=None):
    """ root-mean-square of a vector or rows/columns of a matrix """
    return np.sqrt(np.mean(np.square(abs(x)),axis))



In [ ]:
n_freq = 1024*64
n_time = 512
dc_offset = 0
dc_phase = (1+1j)/np.sqrt(2.)
i_case = 2
H_edge_db = -3
freq = np.linspace(-.5,.5,n_freq,endpoint=False)
tone_enable = 1
dc_reject = 1
x_avg = 2  # =1 for single polarization, =2 for dual
tone_snr_db = 5
tone_freq = np.arange(-.45,.46,.10) 
tone_enable = 1
sw2pm_enable = 0
shear_enable = 1
shear_threshold = 2.3
display_figs = False
special_plot_case = False

In [ ]:
# %%writefile noncoh1.py
# code originally here has been exported
# ditto %%writefile noncoh_plot1.py
# ditto %%writefile noncoh_plot2.py

In [ ]:
%matplotlib inline

# for i_case in range(100):
for i_case in range(50,80):

    # dc_phase = (1+1j)/sqrt(2)
    dc_offset = 0
    dc_reject = 0
    H_edge_db = 0.
    x_avg = 2.  # =1 for single polarization, =2 for dual
    tone_snr_db = 5
    tone_enable = 0
    sw2pm_enable = 0
    shear_enable = 0
    shear_threshold = 2.3
    special_plot_case = False
    n_segment = 1

    if (i_case==1):
        case_str = f'{i_case}: {n_time} x {n_freq}, Single Polarization'
        x_avg = 1.
    elif (i_case==2):
        case_str = f'{i_case}: {n_time} x {n_freq}, Dual Polarization'
    elif (i_case>=10) & (i_case<16):
        dc_offset = .01*(i_case-10)
        case_str = f'{i_case}: {n_time} x {n_freq}, Dual Polarization, DC-offset={dc_offset:.2f}, DC Uncompensated'
    elif (i_case==20):
        dc_offset = .10
        dc_reject = 1
        case_str = f'{i_case}: {n_time} x {n_freq}, Dual Polarization, DC-offset={dc_offset:.2f}, Remove DC'
    elif (i_case>=30) & (i_case<36):
        dc_offset = 0.01 
        dc_reject = 1
        H_edge_db = -1. - 1.*(i_case-30) 
        case_str = f'{i_case}: {n_time} x {n_freq}, Dual Polarization, DC-offset={dc_offset:.2f}, H-Edge={H_edge_db:.1f} dB, Remove DC'
    elif (i_case>=40) & (i_case<46):
        dc_offset = .01
        dc_reject = 1
        sw2pm_enable = 1
        H_edge_db = -1. - 1.*(i_case-40)
        case_str = f'{i_case}: {n_time} x {n_freq}, Dual Polarization, DC={dc_offset:.2f}, H-Edge={H_edge_db:.1f} dB, Remove DC, sw2pm'
    elif (i_case>=50) & (i_case<56):
        dc_offset = 0.01 
        dc_reject = 1
        tone_enable = 1
        H_edge_db = -1. - 1.*(i_case-50) 
        case_str = f'{i_case}: {n_time} x {n_freq}, Dual Polarization, DC={dc_offset:.2f}, H-Edge={H_edge_db:.1f} dB, Remove DC'
    elif (i_case>=60) & (i_case<66):
        dc_offset = .01
        dc_reject = 1
        tone_enable = 1
        sw2pm_enable = 1
        H_edge_db = -1. - 1.*(i_case-60) 
        case_str = f'{i_case}: {n_time} x {n_freq}, Dual Polarization, DC={dc_offset:.2f}, H-Edge={H_edge_db:.1f} dB, Remove DC, sw2pm'
    elif (i_case>=70) & (i_case<76):
        dc_offset = .01
        dc_reject = 1
        tone_enable = 1
        sw2pm_enable = 1
        shear_enable = 1
        H_edge_db = -1. - 1.*(i_case-70)
        case_str = f'{i_case}: {n_time} x {n_freq}, Dual Polarization, DC={dc_offset:.2f}, H-Edge={H_edge_db:.1f} dB, Remove DC, sw2pm Shear {shear_threshold:.2f}'
    else:
        continue

    print(f'\n{case_str}')

    if (abs(H_edge_db-(-4))<.01):
        special_plot_case = True
        
    # run script to generate spectrograms and do evaluation
        
    %run -i ./noncoh1.py
    
    # plot psd original and normalized
    
    special_plot_case = False
    %run -i ./noncoh_plot1.py

    # plot non-coherent gain over all averages
    
    %run -i ./noncoh_plot2.py
        
    

 

In [ ]:
whos

In [ ]:
# Beep in WSL
os.system("powershell.exe '[console]::beep(261.6,700)'")